# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [26]:
# your code here

orders = pd.read_csv('Orders.csv')

orders.head()

orders.shape

(397924, 14)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [29]:
# your code here


customer_table=orders.groupby('CustomerID').agg({'amount_spent':'sum'}) #total amount spent per unique customer

#95% percentile of total amount spent per customer
q95=orders.groupby('CustomerID').agg({'amount_spent':'sum'})['amount_spent'].quantile(.95) 

#75% percentile of total amount spent per customer
q75=orders.groupby('CustomerID').agg({'amount_spent':'sum'})['amount_spent'].quantile(.75)


In [34]:
#VIP column, True if the customer is VIP
customer_table['VIP']=[a>q95 for a in customer_table['amount_spent']] 


#Preferred column, True if the customer is Preferred
customer_table['Preferred']=[q95>=a>q75 for a in customer_table['amount_spent']]


customer_table.head()

,amount_spent,VIP,Preferred
CustomerID,,,
12346,77183.60,True,False
12347,4310.00,False,True
12348,1797.24,False,True
12349,1757.55,False,True
12350,334.40,False,False


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [42]:
# your code here



table_vip_per_country=pd.merge(
    left=orders,
    right=customer_table,
    how="inner",
    on='CustomerID').groupby('Country').agg({'VIP':'sum'})

table_vip_per_country[table_vip_per_country['VIP']==table_vip_per_country['VIP'].max()]


,VIP
Country,
United Kingdom,84185


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [44]:
# your code here


table_vip_pref_per_country=pd.merge(
    left=orders,
    right=customer_table,
    how="inner",
    on='CustomerID').groupby('Country').agg({'VIP':'sum','Preferred':'sum'})


table_vip_pref_per_country['combined']=table_vip_pref_per_country['VIP']+table_vip_pref_per_country['Preferred']

table_vip_pref_per_country[table_vip_pref_per_country['combined']==table_vip_pref_per_country['combined'].max()]

,VIP,Preferred,combined
Country,,,
United Kingdom,84185,137450,221635
